# Problem Set 3 - Empirical IO
## Luiza Zardin

In [1]:
import numpy as np
import pandas as pd
import csv
from numpy import log as log
from scipy.optimize import minimize

In [2]:
#Getting the data
data = pd.read_csv('data_ps2.txt',delimiter='\t',header=None)
data.columns = ['car', 'year', 'firm', 'price', 'quantity', 'weight', 'hp', 'ac', 'nest3', 'nest4']

# Part 1: Computing the HZ Model

In [2]:
def logsumexp(X): # x is a 3-d matrix, len(x_grid) x len(lim_seq) x len([0, 1])
    A = np.amax(X, axis=2)
    return np.log(np.sum(np.exp(X - A[:,:,None]), axis=2)) + A

In [25]:
#Importing the data
data = pd.read_csv('rustdata1.csv')

Calculate (analytically) the gradient of the log-likelihood function in Rust with respect to the parameters of the model and write down the analytic results.

The Rust (1987) problem is: $$\log\mathcal{L} = \sum_t^{T}\log\mathbb{P}\left.\Big[i_t \,\middle|\, x_t\Big]\right.$$ where $\theta = [\theta_c, RC]$ and $\theta_p$ is estimated outside of the ML estimation.
The expected continuation value given today's mileage and replacement decision, $EV\Big(x_t, i_t\Big)$, can be found solving for a fixed point:
$$EV\Big(x_t, i_t\Big) = \int \log\Big(\sum_{i'=0,1}\exp\Big(u(x',i') + \beta \cdot EV\Big(x', i'\Big)\Big)\Big) p(x'|\theta_p, x_t, i_t)$$
With $EV\Big(x_t, i_t\Big)$, I can find: $$\tilde{V}\Big(x_t, i_t\Big)=\left\{\begin{matrix}
-c(0) -RC + \beta EV\Big(x_t, 1\Big) &amp; \text{if } i_t = 1 \\ 
-c(x_t) + \beta EV\Big(x_t, 0\Big) &amp; \text{if } i_t = 0
\end{matrix}\right.$$ And using (4.13) from Rust (1987): $$\mathbb{P}\left.\Big[i_t \,\middle|\, x_t\Big]\right. = \frac{\exp\Big(\tilde{V}\Big(x_t, i_t\Big)\Big)}{\exp\Big(\tilde{V}\Big(x_t, 0\Big)\Big)+\exp\Big(\tilde{V}\Big(x_t, 1\Big)\Big)}$$ This probability takes $\theta_c$, $RC$, $\beta$ and $\theta_p$ as given.
The derivatives are: $$\frac{\partial \log\mathbb{P}}{\partial \theta} = \frac{1}{\mathbb{P}}\frac{\partial \mathbb{P}}{\partial \theta}$$$$\frac{\partial \mathbb{P}}{\partial \theta}\Big[x_t, i_t\Big] = \frac{\exp\Big(\tilde{V}\Big(x_t, 0\Big)+\tilde{V}\Big(x_t, 1\Big)\Big)}{\Big(\exp\Big(\tilde{V}\Big(x_t, 0\Big)\Big)+\exp\Big(\tilde{V}\Big(x_t, 1\Big)\Big)\Big)^2}\Big(\frac{\partial \tilde{V}}{\partial \theta}\Big(x_t, i_t\Big)\Big)-\frac{\partial \tilde{V}}{\partial \theta}\Big(x_t, 1-i_t\Big)\Big)\Big)$$ We could use the implicit function theorem to get the derivative of $EV$, but this will lead to a functional equation.

# Part 2: Estimation MLE and MPEC

Estimate the model using the NPMLE approach of Rust. You will want to use the gradient.

1. Compute the transition probabilities in a separate first stage. You should have 5 of them.

In [91]:
deltamax = data['x_t'].diff().max()
deltas = data['x_t'].diff()[data['x_t'].diff()>0]
k = 5
n = np.size(deltas,0)
limit = np.linspace(0,np.ceil(deltamax),k+1)
theta_3 = np.zeros((k,1))
for i in range(k):
    theta_3[i] =  np.sum((deltas>=limit[i])&(deltas<limit[i+1]))/n

2. Compute $EV (x;\theta)$ for a given guess of the parameters via the fixed point.

In [ ]:
q = 1 # number of parameters in the following function
def c(x, theta_1): # currently defined as a linear function, which apparently Rust preferred
    return theta_1 * x

# Define profit function
def pi(x, i, theta_1, RC): # x is an array, i is not
    if i == 0:
        pi = -c(x, theta_1)
    if i == 1:
        pi = -(RC - c(np.zeros(x.shape), theta_1))
    return pi

# Compute EV(x, θ) via the fixed point
l = 4 # how fine the discretized x grid is, l=1 is as fine as Δy from lim_seq, l=2 is twice as fine, etc.
def EV_fp(theta_1, RC):
    x_max = 14 # maximum value of x - in data, nothing above 13.8...
    Delta_y = lim_seq[1] # Δy - the size of the increments in lim_seq
    x = np.linspace(0, x_max, x_max/Delta_y*l+1) # need such that it includes the addition of Δy, which is characterized by lim_seq
    maxx = np.amax(x) # maximum value in the discretized x grid
    i = np.array([0, 1])
    
    # Construct the payoffs
    Deltay = np.tile(lim_seq, (len(x), 1))
    xplusDeltay = np.tile(x, (len(lim_seq), 1)).T + Deltay # matrix where each column is x + multiple of Δy
    xplusDeltay = np.where(xplusDeltay <= maxx, xplusDeltay, maxx) # keep the highest values of x within the specified domain
    idx = np.tile(np.arange(len(x)), (len(lim_seq), 1)).T + np.arange(len(lim_seq)) * l
    idx = np.where(idx <= len(x) - 1, idx, len(x) - 1) # index used to differentially move up columns
    pi0 = pi(xplusDeltay, 0, theta_1, RC)
    pi1 = pi(xplusDeltay, 1, theta_1, RC)
    
    # Construct p(x_t+1 | i=1)
    pr_i1 = np.vstack((np.ones(len(lim_seq)), np.zeros((len(x) - 1, len(lim_seq)))))
    
    # Initialize the loop
    init_EV = np.zeros((len(x), len(i)))
    EV_tau = init_EV
    err = 1
    err_tol = 1e-12
    iter_num = 1
    iter_lim = 5000
    while err > err_tol and iter_num < iter_lim:
        pdv0 = pi0 + beta * (np.tile(EV_tau[:, 0], (len(lim_seq), 1)).T)[idx, np.arange(len(lim_seq))]
        pdv1 = pi1 + beta * (np.tile(EV_tau[:, 1], (len(lim_seq), 1)).T)[idx, np.arange(len(lim_seq))]
        logsumexp_prepr = logsumexp(np.dstack((pdv0, pdv1)))
        EV_tau1_i0 = np.sum(logsumexp_prepr * transition_pr, axis=1)
        EV_tau1_i1 = np.sum(logsumexp_prepr * pr_i1, axis=1)
        EV_tau1 = np.vstack((EV_tau1_i0, EV_tau1_i1)).T
        err = np.amax(np.abs(EV_tau1 - EV_tau))
        EV_tau = EV_tau1
        iter_num += 1
    if iter_num == iter_lim:
        print("EV didn't converge.")
    return x, EV_tau

# Define EV function
def EV_fct(EV_fp, x_space, x, i): # x is an array, i is not
    return np.interp(x, x_space, EV_fp[:,i]) # interpolate 

# Define choice-specific value function
def v(x, i, theta_1, RC, EV, x_space): # x is an array, i is not
    v = pi(x, i, theta_1, RC) + beta * EV_fct(EV, x_space, x, i)
    return v

# Construct CCP given EV(x, θ)
def CCP(x, i, theta_1, RC):
    x_space, EV = EV_fp(theta_1, RC)
    V1 = v(x, 1, theta_1, RC, EV, x_space)
    V0 = v(x, 0, theta_1, RC, EV, x_space)
    Vi = V0*(i == 0) + V1*(i == 1)
    pr = np.divide(np.exp(Vi), np.exp(V0) + np.exp(V1))
    return pr

# Construct the likelihood
def loglikelihood(theta, x, i):
    theta_1 = theta[:-1]
    RC = theta[-1]
    logl = np.sum(np.log(CCP(x, i, theta_1, RC)))
    return -logl # negative because we are using a minimizer, but it's MLE

# Construct likelihood's gradient
def loglikelihood_grad(theta, x, i): # must accept same arguments as likelihood()
    return 0 # need to compute this by hand (and write it in question 1)

# Solve via MLE
init_guess = np.zeros(q + 1)
# res = minimize(loglikelihood, init_guess, args=(data['x_t'], data['d_t']), method='BFGS', jac=loglikelihood_grad)

Estimate the model using the MPEC method of Su and Judd.

In [ ]:

a = np.ones(3)
print(a)
b = np.zeros((5,3))
print(np.vstack((a, b)))

Compare the results in a table, including the nonparametric answers below and discuss the results.

# Part 3: Stata

This is taken from Han Hong’s problem set at Stanford, the idea is that we can use the arguments in Hotz-Miller (1993), or Pesendorfer Schmidt-Dengler (2008) to construct an optimization free method to recover the utility pa- rameters in the Rust problem.
We begin by defining the choice specific value function with $\varepsilon_{it}$ i.i.d. and EV.
$$v(x,d) = u(x,d) + \beta \int \log \left( \sum_{d' \in D} \exp(v(x', d')) \right) p(x'|x, d) \mathrm{d}x'$$$$v(x,d) = u(x,d) + \beta \int \log \left( \sum_{d' \in D} \exp(v(x', d') - v(x', 1)) \right) p(x'|x, d) \mathrm{d}x' + \beta \int v(x,1)p(x'|x,1) \mathrm{d}x'$$
1. Estimate $p(x′|x,d)$ non parametrically or parametrically (for example as a set of multinomial with $n$ outcomes or an exponential distribution). Call your estimate $\hat{p}(x′|x,d)$.

2. Estimate $p(d|x)$ (the CCP) non-parametrically. You can use the binomial logit model with a basis function (increasing number of terms) or you can use a kernel such as ksdensity or ecdf.

In [3]:
#Calculating market shares
data['share'] = data['quantity']/(1e8)

#Constructing the BLP instruments
for index, row in data.iterrows():
    #mean of characteristics of competing firms in a giving year
    ind = ((data['year'] == data.loc[index,'year']) & (data['firm']!=data.loc[index,'firm']))
    data.loc[index,'z_compw'] = np.mean(data.loc[ind,'weight'])
    data.loc[index,'z_comphp'] = np.mean(data.loc[ind,'hp'])
    data.loc[index,'z_compac'] = np.mean(data.loc[ind,'ac'])
    
#For later renormalization of the coefficients
factors = (-np.mean(data['price']),np.mean(data['weight']),np.mean(data['hp']))

#Normalizing weight, hp and price
data['weight'] = data['weight']/np.mean(data['weight'])
data['hp'] = data['hp']/np.mean(data['hp'])
data['price'] = data['price']/np.mean(data['price'])

#Constructing the LHS variable for the regression
for year in data.year.unique():
    index = data['year']==year
    data.loc[index,'share0'] = 1 - np.sum(data.loc[index ,'share'])

data['y'] = log(data['share']) - log(data['share0'])

I estimate $\hat{\theta}$ using the following GMM procedure: 
$$\max_\theta Q_n\big(\theta \big)$$ 

where $Q_n\big(\theta\big) = -\frac{1}{2}g_n\big(\theta\big)'\,\hat{W}\,g_n\big(\theta\big)$ and 

$g_n\big(\theta\big) = \frac{1}{n}\sum_{r=1}^R g\big(w_r; \theta\big)$.

$g\big(w_r; \theta\big) = z_r \cdot (y_r - x_r \cdot \theta)$ is the residual $\xi_{jt}$ from the model above times the instruments calculated using $w_r$, a row of data.

The efficient GMM is estimated with weighting matrix $\hat{W}=\left(\frac{1}{n} \sum_{o=1}^n{\Big(g(w_o,\theta_{first})-g_n(w,\theta_{first})\Big) \cdot \Big(g(w_o,\theta_{first}) - g_n(w,\theta_{first})\Big)'}\right)^{-1}$

In [4]:
# defining GMM objects
# we have n observations, m regressors and k instruments
# y is a nx1 vectorm
# x is a nxm matrix
# z is a nxk matrix

def gn(y,x,z,theta):
    return z.T * (y - x * theta.reshape((x.shape[1],1)))
     
# Optimal GMM weighting matrix
def Wmtrx(y,x,z,theta):
    result = np.zeros((z.shape[1],z.shape[1]))
    for i in range(y.shape[0]):
        result += gn(y[i,:],x[i,:],z[i,:],theta) * gn(y[i,:],x[i,:],z[i,:],theta).T
    result = 1/y.shape[0] * result
    return np.matrix(result).I

# GMM objective function (as a function of the parameters of the model, for given data)
def Qn(theta,y,x,z,W):
    return 1/2 * gn(y,x,z,theta).T * W * gn(y,x,z,theta)

# GMM using optimal weighting matrix
def GMM(y,x,z,guess):
    W = np.eye(z.shape[1])
    res = minimize(Qn,guess,args=(y,x,z,W),method='Nelder-Mead')
    W = Wmtrx(y,x,z,res.x)
    return minimize(Qn,res.x,args=(y,x,z,W),method='Nelder-Mead')

In [5]:
# Estimating GMM for each year
m = 4
coefs = np.zeros((data.year.unique().shape[0],m))

for i, year in enumerate(data.year.unique().tolist()):
    
    index = data['year']==year
    
    y = np.matrix(data.loc[index,'y']).T
    x = np.matrix(data.loc[index,['price','weight','hp','ac']])
    z = np.matrix(data.loc[index,['z_compw','z_comphp','z_compac','weight','hp','ac']])
    
    tguess = np.matrix(np.zeros([1,x.shape[1]]))
    coefs[i,:] = GMM(y,x,z,tguess).x
    

In [6]:
#Printing coefficients
pd.DataFrame(data=coefs[:,:],index=data.year.unique(),columns=['price (-alpha)','weight','hp','ac']) 

,price (-alpha),weight,hp,ac
1990,14.372416,0.063108,-19.703589,-3.023163
1991,9.205850,-1.854889,-14.487028,-1.100564
1992,6.932062,-2.469909,-12.324078,-0.489103


## Question 4

**Explain why own and cross price elasticites from this logit model may be unrealistic.**

In the logit model, own price elasticities are increasing in price, which is somewhat unrealistic (we would think people who buy
expensive products are less sensitive to price). Also, cross price elasticities depend only on market shares and prices but not on similarities between goods (IIA property), which is also unrealistic.

# Nested Logit 

## Question 5

**Write down market share for a single product as a function of the vector $\delta^*$ and the nesting parameter $\sigma$. Use the $\sigma$ and group notation used by Berry (1994 Rand), not the $\lambda$ notation used by Train and McFadden.**

We have that $s_{jt} = s_{jt|g} s_g$. Using the logit formula we get that:
$$s_{jt} = \frac{\exp\left(\frac{\delta_{jt}^*}{1 - \sigma}\right)}{\left(\sum_{k \in \mathcal{J}_g} \exp\left(\frac{\delta_{kt}^*}{1 - \sigma}\right)\right)^{\sigma} \left(\sum_h \left(\sum_{k \in \mathcal{J}_h} \exp\left( \frac{\delta_{kt}^*}{1 - \sigma} \right) \right)^{1 - \sigma}\right)}.$$

**Invert that equation to solve for $\delta_j^*$ as a function of market shares, within group shares, and $\sigma$.**

For the outside good we have:
$$s_{0t} = \frac{1}{\sum_g \left(\sum_{j \in \mathcal{J}_g} \exp\left( \frac{\delta_{jt}^*}{1 - \sigma} \right) \right)^{1 - \sigma}}.$$
Taking log's and differences we get:
$$\log s_{jt} - \log s_{0t} = \frac{\delta_{jt}^*}{1-\sigma} - \sigma \log{D_{gt}}$$
where $D_{gt}=\sum_{j \in g} e^{\delta_{jt}^*/(1-\sigma)}$. That yields:
$$\log s_{jt} - \log s_{0t} = x'_{jt}\beta_t - \alpha_t p_{jt} + \sigma \log s_{jt|g} + \xi_{jt}  $$

## Question 6

**Estimate the demand system parameters using just the demand-side moment conditions for this nested-logit model. Instrument for price using the average of product characteristics (i.e. weight, hp, ac) of products produced by other firms in a given year. Similarly, construct the instrumental variables by using the average of product characteristics of other products (including those of the same firm) within the same group in a given year. For your nests, use two different nesting structures: one that groups products based on the “nest3” variable, and one that groups products based on the “nest4” variable. (Implicitly, the outside good is in its own additional group in each year.) Calculate 2SLS estimates, allowing all coefficients to vary across the three years of data, and use them as starting values for the optimization in your GMM routine. Report your results for $n = 3$ (inside) groups and $n = 4$ (inside) groups.**

In [7]:
nestnms = ['nest3', 'nest4']
sharesnms = ['share_n3', 'share_n4']
instrnms = [['z_compw_n3','z_comphp_n3','z_compac_n3'],['z_compw_n4','z_comphp_n4','z_compac_n4']]

for n in range(2):

    #Constructing within group shares
    for i, year in enumerate(data.year.unique().tolist()):
        for j, nest in enumerate(data.eval(nestnms[n]).unique().tolist()):
            index = ((data['year']==year) & (data[nestnms[n]]==nest))
            qnt = sum(data.loc[index, 'quantity'])
            data.loc[index, sharesnms[n]] = log(data.loc[index, 'quantity']/qnt)

    #Constructing within group instruments
    for index, row in data.iterrows():
        #mean of characteristics of competing firms in a giving year
        ind = ((data['year'] == data.loc[index,'year']) & (data[nestnms[n]] == data.loc[index,nestnms[n]])
               & (data['car'] != data.loc[index,'car']))
        data.loc[index,instrnms[n][0]] = np.mean(data.loc[ind,'weight'])
        data.loc[index,instrnms[n][1]] = np.mean(data.loc[ind,'hp'])
        data.loc[index,instrnms[n][2]] = np.mean(data.loc[ind,'ac'])

In [8]:
# Calculate the 2SLS and GMM estimates for the Nested Logit
k = 0
m = 5
coefs = np.zeros((2*data.year.unique().shape[0],m))

for i, year in enumerate(data.year.unique().tolist()):
    for j, nest in enumerate(nestnms):
        index = data['year']==year
        y = np.matrix(data.loc[index, 'y']).T
        x = np.matrix(data.loc[index, ['price','weight','hp','ac',sharesnms[j]]])
        z = np.matrix(data.loc[index, instrnms[j] + ['z_compw','z_comphp','z_compac','weight','hp','ac']])
        t2SLS = (x.T * z * (z.T * z).I * z.T * x).I * x.T * z * (z.T * z).I * z.T * y
        coefs[k,:] = GMM(y,x,z,t2SLS).x
        k += 1
        

In [9]:
k = 0
labels = []
for i, year in enumerate(data.year.unique().tolist()):
    for j, nest in enumerate(nestnms):
        labels.append(str(year)+','+nest)
        
pd.DataFrame(data=coefs[:,:],index=labels,columns=['price (-alpha)','weight','hp','ac','sjtg (sigma)']) 

,price (-alpha),weight,hp,ac,sjtg (sigma)
"1990,nest3",-2.407367,-1.818427,0.416030,1.827852,1.104780
"1990,nest4",1.929430,-0.956090,-4.849609,0.503870,0.950700
"1991,nest3",0.125390,-2.017619,-1.362030,0.502705,1.041927
"1991,nest4",2.641000,0.166342,-6.703076,-0.053670,0.930611
"1992,nest3",0.604341,-2.153633,-1.792808,0.250344,0.982752
"1992,nest4",2.510011,-0.202878,-6.484872,-0.172786,0.849344


**b) Suppose instead that we estimated a version of a nested-logit model that pooled all three years’ worth of data. What assumption on intertemporal substitution patterns is implicit in this choice?**

We need to assume that the conditional error is independent over time. In practice, because we have unobserved quality components in the errors and this are very likely to be persistent, this assumption would be incorrect.


**c) Are your estimates of sigma sensitive to the number of groups? Can you give an explanation for this result?**

Yes, $\sigma$ is usually higher with three groups than with four groups. $\sigma$ captures the within group correlation in market shares. I would expect to have higher within group correlation the finer the nest structure, but maybe the groups in the nests with 4 groups are not very well defined.

**d) How does your estimate of $\alpha$ change across the three years' of data?**

$\alpha$ varies a lot over the years (even from negative to positive) and it appears to be increasing over the years for a given nest structure. With four nests we get higher $\alpha$'s.

## Question 7

**a) How is nested logit an improvement over plain logit?**

The nested logit allows for a more flexible substitution pattern. The IIA property now holds only for products within the same group and not across all goods. Within each group we have standard logit, but products in different nests have less in common, and therefore are not as good substitutes.

**b) Think of another nesting structure you would use, and explain what additional data you would need to estimate it.**

We could use multiple levels of nesting or potentially overlapping nests, but this would require a modification of the equations used before.


**c) Why might all forms of nested logit be problematic?**

One drawback with nested-logit is that we need to a-priori classify the products, so if the groups are not well defined we'll have the same propblems as in the logit.

## Question 8

**Suppose we were interested in improving the substitution patterns.**

**a) Would the Multinomial Probit model be appealing in this setting? Why, or why not?**

The probit replaces the assumption that the errors are EV and i.i.d. with the assumption that the errors are normally distributed but with an arbitrary covariance matrix. This allows for very flexible substitution patterns but creates a huge number of parameters to estimate in the covariance matrix. Usually to make it operational, one has to make assumptions in the covariance matrix, which are as arbitraty as choosing nests.

**b) Would the Pure Characteristics model be appealing in this setting? Why, or why not?**

The pure characteristics model assumes that there is no unobserved product specific characteristics, so it could not account for a BWM having a higher market share than other cars with similar observed characteristics and lower price. It would not be appealing in this setting.

# Mixed Logit

## Question 9

**Now assume that the model is a logit model but each individual has a different price coefficient, i.e.
$$u_{ijt} = x_{jt}\beta - \alpha_i p_{jt} + \xi_{jt} + \varepsilon_{ijt}.$$**

**a) Suppose $\alpha_i = 1 / y_i$ and $y_i$ is distributed lognormally. Write out the moment conditions and estimation algorithm you would use to estimate this model.**

We have that $\alpha_i = 1 / y_i$, and $\log(y_i) \sim \mathcal{N}(\mu_y, \sigma_y^2)$. Note that if $Y = 1 / X$, then $\log(Y) = \log(X^{-1}) = -\log(X)$, so if $\log(X) \sim \mathcal{N}(\mu, \sigma^2)$, then $\mathbb{E}[Y] = \exp(-\mu + \frac12 \sigma^2)$ by the formula for the expectation of a log normally-distributed random variable. We can therefore derive the following mean utility:
$$\delta_{jt}^* = x_{jt}\beta - \exp\left(-\mu_y + \frac12 \sigma_y^2\right) p_{jt} + \xi_{jt}.$$
Note that because $\varepsilon_{ijt} \overset{iid}{\sim} EV(I)$, we can write the logit shares as follows:
$$\begin{align}
s_{jt} &= \int_0^\infty \frac{\exp\left(x_{jt} \beta - \frac{1}{y_i} p_{jt} + \xi_{jt} \right)}{1 + \sum_k \exp\left(x_{kt} \beta - \frac{1}{y_i} p_{kt} + \xi_{kt} \right)} \mathrm{d}F(y_i) \\
&= \int_0^\infty \frac{\exp\left(x_{jt} \beta - \frac{1}{y_i} p_{jt} + \xi_{jt} \right)}{1 + \sum_k \exp\left(x_{kt} \beta - \frac{1}{y_i} p_{kt} + \xi_{kt} \right)} \frac{1}{y_i \sigma_y \sqrt{2 \pi}} \exp\left( -\frac{(\log(y_i) - \mu_y)^2}{2\sigma_y^2} \right) \mathrm{d}y_i \\
&= \int_0^\infty \frac{\exp\left(\delta_{jt}^* + \left(\exp\left(-\mu_y + \frac12 \sigma_y^2\right) - \frac{1}{y_i}\right) p_{jt} \right)}{1 + \sum_k \exp\left(\delta_{kt}^* + \left(\exp\left(-\mu_y + \frac12 \sigma_y^2\right) - \frac{1}{y_i}\right) p_{kt} \right)} \frac{1}{y_i \sigma_y \sqrt{2 \pi}} \exp\left( -\frac{(\log(y_i) - \mu_y)^2}{2\sigma_y^2} \right) \mathrm{d}y_i \\
&= \int_{-\infty}^\infty \frac{\exp\left(\delta_{jt}^* + \left(\exp\left(-\mu_y + \frac12 \sigma_y^2\right) - \exp(-\psi_i)\right) p_{jt} \right)}{1 + \sum_k \exp\left(\delta_{kt}^* + \left(\exp\left(-\mu_y + \frac12 \sigma_y^2\right) - \exp(-\psi_i)\right) p_{kt} \right)} \frac{1}{\sigma_y \sqrt{2 \pi}} \exp\left( -\frac{(\psi_i - \mu_y)^2}{2\sigma_y^2} \right) \mathrm{d}\psi_i,
\end{align}$$
where the final line follows from setting $\psi_i := \log(y_i)$. This gives us the following moment condition (assuming we are not pooling across years):
$$\mathbb{E}[g(w_{lt}, \theta_0)] = \mathbb{E}\left[z_{lt} \left(\delta_{lt}^* - x_{lt}\beta + \exp\left(-\mu_y + \frac12 \sigma_y^2\right) p_{lt} \right)\right] = 0,$$
where $z_{lt}$ are our instruments for price. This moment condition allows us to use GMM. We do not observe mean utility, however; therefore, we propose a system based on BLP. Let
$$g_n(\theta) = \frac{1}{n} \sum_{l=1}^n \left[ z_{l}' \left(\delta_{l}^* - x_{l}\beta + \exp\left(-\mu_y + \frac12 \sigma_y^2\right) p_l \right) \right]$$

where $\theta=\Big[\beta, \mu_y, \sigma_y\Big]$.

Use the following algorithm to obtain an estimate $\hat{\theta}$. (*Note: Time subscripts have been dropped, but if we wish to estimate individual years, we can repeat the following algorithm for the data separated by year.*)

**Step 1:** Take an initial value of $\hat{\theta} \in \Theta$ and an initial weighting matrix (e.g. identity matrix).

**Step 2:** Use the following contraction mapping proposed by BLP to solve for $\delta$ such that $s_{j}(\delta, \theta) = \tilde{s}_{j}$ (i.e. that observed shares and predicted shares are identical):
$$\delta^{(k)}(\theta) = \delta^{(k-1)}(\theta) + \log(\tilde{s}_j) - \log(s_j(\delta^{(k-1)}, \theta)).$$
Iterate on this equation using beginning with an initial guess of $\delta$ (make sure it's reasonable--perhaps start with one of the previous $\hat{\delta}$ values) until $\left\vert \delta^{(k)}(\theta) - \delta^{(k-1)}(\theta)\right\vert < 10^{-13}$. To compute the numerical integral, we can use a Gauss-Hermite approximation (which is very fast) but we need to give it a sufficiently large number of points so that it is accurate and precise. (*Note: We are assuming here that the contraction mapping holds in this case. BLP proved that it holds in the normally distributed $\beta$ case, so it might not hold in this inverse log-normally distributed $\alpha$ case.*)

**Step 3:** Evaluate objective function: $-\frac12 g_n(\theta)' \hat{W}_n g_n(\theta)$.

**Step 4:** Choose next $\hat{\theta}' \in \Theta$ in systematic way to search over $\Theta$ to maximize objective function. Return to Step 2 with $\hat{\theta}'$. Repeat Steps 2 - 4 until convergence (i.e., we have reached the maximum).

**Step 5:** Update the weighting matrix with the estimated $\hat{\theta}$ that minimizes the objective function and repeat Steps 1 through 4 (2-stage efficient GMM).

This returns an estimate $\hat{\theta}_{BLP}$.

**b) Now suppose $\alpha_i = \alpha_1 + \alpha_2 / y_i$ and $y_i$ is still distributed lognormally. How exactly would this change the estimation? Write out the moment conditions and estimation algorithm you would use to estimate $\beta$ in this model. Are the parameters of the lognormal distribution, $\alpha_1$ and $\alpha_2$ identified by the data provided to you?**

$\alpha_i = \alpha_1+\alpha_2/y_i \implies \mu_\alpha = \alpha_1+\alpha_2 \cdot \exp(-\mu_{y}+\frac{1}{2}\sigma_{y}^{2})$ hence $\alpha_1$ is not identified, $\alpha_2$ might be by the variance or higher moments. The moment conditions would be derived by plugging the new $\mu_\alpha$ into the integral above.

## Question 10

**a) Among the other parameters you would have estimated in question 9a are the mean and the variance of the lognormal distribution. Now assume that you knew that the mean of income was \$35,000 and that the standard deviation is \$45,000. Using only the demand system, estimate the $\beta$ parameters under these assumptions. Continue to use the moment conditions involving the excluded instruments you used for plain logit.**

With fixed parameters on the distribution of $y_i$ there are only 3 parameters to estimate. $$35000 = \exp(\mu_y + \frac{\sigma_y^2}{2})$$$$45000^2 = (\exp(\sigma_y^2) - 1)\cdot\exp(2\mu_y+\sigma_y^2)$$ Hence: $$2(\log 35000 - \mu_y) = \sigma_y^2$$$$2\log45000 = \log(\exp(\sigma_y^2) - 1) + 2\mu_y+\sigma_y^2$$ And: $$\mu_y = \log\Big(\frac{35000^2}{\sqrt{45000^2 + 35000^2}}\Big)$$$$\sigma_y = \sqrt{\log\Big(1+\frac{45000^2}{35000^2}\Big)}$$

In order to estimate $\beta$, we need to be able to evaluate $\tilde{s}_j$. We will use Gauss-Hermite quadrature because it is quick to compute with a large number of weights to ensure precision and accuracy. Note:
$$\begin{align}
s_j &= \int_{-\infty}^\infty \frac{1}{\sigma_y \sqrt{2\pi}} \exp\left(-\frac{(\psi_i - \mu_y)^2}{2\sigma_y^2}\right) h(\psi_i) \mathrm{d}\psi_i \\
&= \int_{-\infty}^\infty \frac{1}{\sqrt{\pi}} \exp(-x_i^2) h(\sqrt{2}\sigma_y x_i + \mu_y) \mathrm{d}x_i \qquad \text{where } x_i = \frac{\psi_i - \mu_i}{\sqrt{2}\sigma_y} \\
&\approx \frac{1}{\sqrt{\pi}} \sum_i \omega_i h(\sqrt{2}\sigma_y x_i + \mu_y)
\end{align}$$

In [10]:
#Calculates shares by using Gauss-Hermite quadrature
def sj(delta, p, m, theta):
    mu_y = theta[0]
    sigma_y = theta[1]
    P = np.tile(p, (m,1))
    Delta = np.tile(delta, (m,1))
    n = len(p)
    points, weights = np.polynomial.hermite.hermgauss(m)
    points = np.tile(points, (n,1)).T
    weights = np.tile(weights, (n,1)).T
    psi = np.sqrt(2) * sigma_y * points + mu_y
    h_num = np.exp(Delta + np.multiply((np.exp(-mu_y + 1/2 * sigma_y ** 2) - np.exp(-psi)), P))
    h_denom = np.tile(1 + np.sum(h_num, axis=1), (n,1)).T
    h = np.divide(h_num, h_denom)
    s = 1 / np.sqrt(np.pi) * np.sum(np.multiply(weights, h), axis=0)
    return s

#Calculates the deltas
def delta_fp(init_delta, shares, p, theta):
    error = 1
    iteration = 0
    delta = init_delta
    while error > 1e-13 and iteration < 1e3:
        delta_old = delta
        delta = delta_old + np.log(shares) - np.log(sj(delta_old, p, 30, theta))
        iteration += 1
        error = np.amax(np.absolute(delta - delta_old))
    if error > 1e-13:
        print("Fixed point did not converged")
        print("Iterations: " + str(iteration))
        print("Error: " + str(error))
    return delta

mu_y = np.log(35000 ** 2 / (45000**2 + 35000**2) ** (1/2))
sigma_y = np.log(1 + 45000 ** 2 / 35000 ** 2) ** (1/2)
theta = [mu_y, sigma_y]

m = 3
coefs = np.zeros((data.year.unique().shape[0],m))

for i, year in enumerate(data.year.unique().tolist()):
    
    index = data['year']==year
    #calculating delta
    delta_BLP = delta_fp(np.zeros(len(data.loc[index, 'share'])), data.loc[index, 'share'], data.loc[index, 'price'], theta)
    y = np.matrix(delta_BLP + np.exp(-mu_y + 1/2 * sigma_y ** 2) * data.loc[index, 'price']).reshape(delta_BLP.shape[0],1)
    x = np.matrix(data.loc[index, ['weight','hp','ac']])
    z = np.matrix(data.loc[index, ['z_compw','z_comphp','z_compac','weight','hp','ac']])
    #GMM estimation
    init_guess = np.zeros(3)
    coefs[i,:] = GMM(y,x,z,init_guess).x

In [11]:
pd.DataFrame(data=coefs[:,:],index=data.year.unique(),columns=['weight','hp','ac']) 

,weight,hp,ac
1990,-7.379300,-1.388828,2.570353
1991,-7.069365,-1.164322,1.229033
1992,-7.105062,-0.915393,0.968160


**b)  Are the own and cross price elasticities from this system more realistic than those in the plain and nested logit models, and if so why? One way to evaluate this is to produce the $J$ matrix of diversion ratios, as well as reporting the median own price elasticity.**

The mixed-logit model imposes less a-priori restrictions on the substitution patterns, mainly because it does not exhibit logit's restrictive independence of irrelevant alternatives (IIA) property. We should get more realistic own and cross price elasticities, as long as the assumption of log-normality is not too crazy.